<img src="https://iteso.mx/documents/27014/202031/Logo-ITESO-MinimoH.png"
     align="right"
     width="300"/>

# Redes Neuronales Convulucionales (CNN) multivariadas para series de tiempo
## *Modelos no lineales para pronósitico*  - Pedro Martinez

---

# CNN 1D multivariadas para pronóstico del clima

Hoy en día, Google y OpenWeather usan variantes de CNN en sus pipelines de nowcasting meteorológico ya que son capaces de detectar patrones temporales locales en series de tiempo, como picos de temperatura o tendencias estacionales.

Usaremos datos reales de la API [Open-Meteo](https://open-meteo.com/) para **predecir la temperatura de las próximas 48 h**, empleando las variables:

| Variable | Descripción | Unidad | Fuente |
|-----------|--------------|---------|---------|
| **Temperatura** | Temperatura del aire a 2 metros | °C (grados Celsius) | `temperature_2m` |
| **Humedad relativa** | Porcentaje de saturación de vapor de agua | % | `relative_humidity_2m` |
| **Nubosidad** | Cobertura nubosa promedio | % | `cloud_cover` |
| **Velocidad del viento** | Velocidad del viento a 10 metros | m/s | `wind_speed_10m` |

---

Una CNN aplicada a series de tiempo multivariables procesa una secuencia de observaciones:

$$
X_t =
\begin{bmatrix}
x_{t,1} & x_{t,2} & \dots & x_{t,n}
\end{bmatrix}
\in \mathbb{R}^{n}
$$

donde $n$ es el número de variables de entrada (en este caso, 4).

Si utilizamos una **ventana temporal de tamaño $T$**, la entrada al modelo es:

$$
\mathbf{X} =
\begin{bmatrix}
X_{t-T+1} \\
X_{t-T+2} \\
\vdots \\
X_t
\end{bmatrix}
\in \mathbb{R}^{T \times n}
$$

Cada capa convolucional aplica un conjunto de filtros (*kernels*) que aprenden patrones espacio-temporales en la ventana:

$$
h_j^{(l)}(t) = f \!\left(
\sum_{i=1}^{n}
\sum_{k=0}^{K-1}
w_{ij}^{(l)}(k) \, x_i(t - k)
+ b_j^{(l)}
\right)
$$

donde:

- $w_{ij}^{(l)}(k)$ son los pesos del filtro $j$ en la posición $k$ de la capa $l$.  
- $b_j^{(l)}$ es el sesgo asociado al filtro $j$.  
- $f(\cdot)$ es una función de activación no lineal (típicamente **ReLU**).  
- $K$ es el tamaño del kernel (por ejemplo, 3 horas).

El resultado de esta capa convolucional se **aplasta (*flatten*)** y se conecta a una capa **densa (fully connected)** para generar la predicción final.

---

## Interpretación

- La CNN aprende **patrones locales** (por ejemplo, cómo la humedad y la nubosidad afectan la temperatura en las últimas horas).  
- Al combinar varias variables de entrada, la red actúa como un **modelo multivariable no lineal**.  
- El entrenamiento minimiza una función de pérdida (MSE):


In [10]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import requests
import pandas as pd
import plotly.express as px
import tensorflow as tf
from tensorflow.keras import layers, models, Input
import plotly.graph_objects as go
from datetime import datetime, timedelta, timezone
import pytz


In [11]:
CITY_TZ = "America/Mexico_City"
lat, lon = 20.31, -103.18  # Chapala

tz = pytz.timezone(CITY_TZ)

# --- API Open-Meteo ---
url = (
    "https://api.open-meteo.com/v1/forecast"
    f"?latitude={lat}&longitude={lon}"
    "&hourly=temperature_2m,relative_humidity_2m,cloud_cover,wind_speed_10m"
    "&past_days=15"
    f"&timezone={CITY_TZ}"
)
res = requests.get(url).json()

# --- Construcción del DataFrame ---
df = pd.DataFrame({
    "time": pd.to_datetime(res["hourly"]["time"]),
    "temp": res["hourly"]["temperature_2m"],
    "humidity": res["hourly"]["relative_humidity_2m"],
    "clouds": res["hourly"]["cloud_cover"],
    "wind": res["hourly"]["wind_speed_10m"],
}).set_index("time")

# 🔹 Asegurar zona local
df.index = df.index.tz_localize(tz)

# 🔹 Calcular hora local actual y filtrar HASTA una hora atrás
now_local = datetime.now(tz)
cutoff = now_local - timedelta(hours=1)
df = df[df.index <= cutoff]


In [12]:
fig = px.line(df, y=["temp","humidity","clouds","wind"],
              title="Variables meteorológicas",
              template="plotly_white")
fig.show()

En equipos de 2 personas, realizar las siguientes partes:

Seleccionar variable objetivo

In [13]:
TARGET = "humidity"
FEATURES = ["temp", "clouds", "wind"]

# Mantén solo columnas necesarias y descarta filas con NA
df_ml = df[[TARGET] + FEATURES].dropna().copy()

print(df_ml.tail(3))
print("Shape:", df_ml.shape)

                           humidity  temp  clouds  wind
time                                                   
2025-10-31 15:00:00-06:00        34  25.8      33   2.8
2025-10-31 16:00:00-06:00        32  25.5      27   1.1
2025-10-31 17:00:00-06:00        34  24.9      65   0.5
Shape: (378, 4)


Escalar los datos

In [14]:
from sklearn.preprocessing import MinMaxScaler

# Preparar scalers (se ajustan después del split)
x_scaler = MinMaxScaler(feature_range=(0, 1))
y_scaler = MinMaxScaler(feature_range=(0, 1))

print("Scalers listos. Se ajustarán con TRAIN en el siguiente paso.")


Scalers listos. Se ajustarán con TRAIN en el siguiente paso.


Crear ventanas temporales


*   Se sugiere utilizar 12 o 24 valores anteriores


In [15]:
import numpy as np

def make_windows(X: np.ndarray,
                 y: np.ndarray,
                 window_size: int = 24,
                 horizon: int = 48,
                 step: int = 1):
    """
    Genera pares (ventana, objetivo) para multi-step forecasting.
    - X: array (T, n_features)
    - y: array (T,) o (T, 1)
    - window_size: # de pasos históricos por muestra (p.ej., 24 horas)
    - horizon: # de pasos futuros a predecir (p.ej., 48 horas)
    - step: salto entre ventanas consecutivas

    Return:
      Xw: (num_samples, window_size, n_features)
      Yw: (num_samples, horizon)  # vector multi-step
      idx: índices iniciales (timestamps opcionales si llevas tracking)
    """
    if y.ndim == 2 and y.shape[1] == 1:
        y = y.ravel()

    T = len(y)
    samples = []
    targets = []
    starts = []

    end_limit = T - horizon
    start = window_size

    for i in range(start, end_limit + 1, step):
        x_win = X[i - window_size:i, :]                 # (window_size, n_features)
        y_out = y[i:i + horizon]                        # (horizon,)
        samples.append(x_win)
        targets.append(y_out)
        starts.append(i - 1)  # índice del último punto observado

    if len(samples) == 0:
        return np.empty((0, window_size, X.shape[1])), np.empty((0, horizon)), np.array([])

    Xw = np.stack(samples, axis=0)
    Yw = np.stack(targets, axis=0)
    idx = np.array(starts)

    return Xw, Yw, idx

print("Función make_windows definida (aún no se usa).")


Función make_windows definida (aún no se usa).


Dividir en entrenamiento y prueba

In [16]:
import pandas as pd

# --------------------------------
# Sugerencia de split: últimas 3 d
# --------------------------------
TEST_DAYS = 3
split_time = df_ml.index.max() - pd.Timedelta(days=TEST_DAYS)

df_train = df_ml.loc[:split_time].copy()
df_test  = df_ml.loc[split_time + pd.Timedelta(hours=1):].copy()

print("Rango TRAIN:", df_train.index.min(), "→", df_train.index.max(), "| rows:", len(df_train))
print("Rango TEST :", df_test.index.min(),  "→", df_test.index.max(),  "| rows:", len(df_test))

# -------------------------------
# Ajustar scalers SOLO con TRAIN
# -------------------------------
X_train_raw = df_train[FEATURES].to_numpy(dtype=float)
y_train_raw = df_train[[TARGET]].to_numpy(dtype=float)   # 2D para scaler

x_scaler.fit(X_train_raw)
y_scaler.fit(y_train_raw)

# Transformar TRAIN/TEST
X_train = x_scaler.transform(X_train_raw)
y_train = y_scaler.transform(y_train_raw).ravel()        # 1D para comodidad

X_test  = x_scaler.transform(df_test[FEATURES].to_numpy(dtype=float))
y_test  = y_scaler.transform(df_test[[TARGET]].to_numpy(dtype=float)).ravel()

print("Shapes (X_train, y_train):", X_train.shape, y_train.shape)
print("Shapes (X_test , y_test ):", X_test.shape,  y_test.shape)

# Parámetros por defecto sugeridos para luego hacer ventanas
WINDOW_SIZE = 24   # 24 horas de historia
HORIZON     = 48   # predecir 48 horas a futuro


Rango TRAIN: 2025-10-16 00:00:00-06:00 → 2025-10-28 17:00:00-06:00 | rows: 306
Rango TEST : 2025-10-28 18:00:00-06:00 → 2025-10-31 17:00:00-06:00 | rows: 72
Shapes (X_train, y_train): (306, 3) (306,)
Shapes (X_test , y_test ): (72, 3) (72,)


Construir la CNN en Keras
- Recordando las recomendaciones de la clase pasada
- Cambiar número de filtros (16, 32, 64)
- Cambiar tamaño del kernel (2, 3, 5)
- Cambiar función de activación (relu, tanh)
- Cambiar batch_size o epochs

Entrenar el modelo utilizando una semilla para reproducibilidad

Evaluar y visualizar utilizando plotly